In [ ]:
import pandas as pd
weather_set_dirty = pd.read_csv('inCSV/Weather_LosAngeles.csv', sep=",")

In [ ]:
weather_set_dirty['timestamp'] = pd.to_datetime(weather_set_dirty['timestamp'])
mean_values_by_day = weather_set_dirty.groupby(weather_set_dirty["timestamp"].dt.date).mean(numeric_only=True).round(1)

In [ ]:
for col in mean_values_by_day.columns:
    weather_set_dirty[col] = weather_set_dirty.apply(
        lambda row: mean_values_by_day.loc[row['timestamp'].date(), col]
        if pd.isnull(row[col]) else row[col],
        axis=1
    )

In [ ]:
weather_set_dirty['cloud_cover_description'].fillna(method='ffill',inplace=True)
#weather_set_dirty.to_csv('outCSV/Clean_Weather_Set.csv', index=False)

In [ ]:
weather_set_dirty.info()

In [ ]:
bike_set_dirty = pd.read_csv('inCSV/metro_2017_2022.csv', sep=',')

In [ ]:
bike_set_dirty.sort_values('start_station_id')
bike_set_dirty['start_station_lat'].fillna(method="ffill", inplace=True)
bike_set_dirty['start_station_lon'].fillna(method="ffill", inplace=True)

bike_set_dirty.sort_values('end_station_id')
bike_set_dirty['end_station_lat'].fillna(method="ffill", inplace=True)
bike_set_dirty['end_station_lon'].fillna(method="ffill", inplace=True)

In [ ]:
bike_set_dirty_agg_start = bike_set_dirty.groupby('start_station_id')[['start_station_lat', 'start_station_lon']].agg(lambda x: x.mode().iloc[0])
bike_set_dirty_agg_end = bike_set_dirty.groupby('end_station_id')[['end_station_lat', 'end_station_lon']].agg(lambda x: x.mode().iloc[0])

In [ ]:
bike_set_dirty_merge = bike_set_dirty.merge(bike_set_dirty_agg_start, on='start_station_id', suffixes=('','_mode'))
bike_set_dirty_merged = bike_set_dirty_merge.merge(bike_set_dirty_agg_end, on='end_station_id', suffixes=('','_mode'))
bike_set_dirty_merged.info()

In [ ]:
for i, row in bike_set_dirty_merged.iterrows():
    bike_set_dirty_merged.at[i, 'start_station_lat'] = bike_set_dirty_merged.loc[i, 'start_station_lat_mode']
    bike_set_dirty_merged.at[i, 'start_station_lon'] = bike_set_dirty_merged.loc[i, 'start_station_lon_mode']
    bike_set_dirty_merged.at[i, 'end_station_lat'] = bike_set_dirty_merged.loc[i, 'end_station_lat_mode']
    bike_set_dirty_merged.at[i, 'end_station_lon'] = bike_set_dirty_merged.loc[i, 'end_station_lon_mode']

In [ ]:
bike_set_almost_clean = bike_set_dirty_merged
bike_set_almost_clean.info()

In [ ]:
from datetime import datetime
dateformat0 = "%H/%M/%S %d-%m-%Y"
dateformat1 = "%Y-%m-%d %H:%M:%S"
for i, row in bike_set_almost_clean.iterrows():
    try:
        bike_set_almost_clean.at[i, 'start_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'start_time'], dateformat0)
    except ValueError:
        bike_set_almost_clean.at[i, 'start_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'start_time'], dateformat1)
    try:
        bike_set_almost_clean.at[i, 'end_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'end_time'], dateformat0)
    except ValueError:
        bike_set_almost_clean.at[i, 'end_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'end_time'], dateformat1)
bike_set_almost_clean.head()

In [ ]:
bike_set_clean = bike_set_almost_clean.drop(['start_station_lat_mode', 'start_station_lon_mode', 'end_station_lat_mode', 'end_station_lon_mode'], axis=1)
bike_set_clean.info()

In [ ]:
bike_set_clean['start_time'] = pd.to_datetime(bike_set_almost_clean['start_time'], format="%Y-%m-%d %H:%M:%S")
bike_set_clean.info()

In [ ]:
bike_set_clean['end_time'] = pd.to_datetime(bike_set_almost_clean['end_time'], format="%Y-%m-%d %H:%M:%S")
bike_set_clean.info()

In [ ]:
bike_set_clean.to_csv('outCSV/Clean_Metro_Set.csv')